## 1) Load the Microsoft.com data into HDFS

delete data folder and re-create it

In [1]:
%%bash
hdfs dfs -rm -r /user/stud22/ex2
hdfs dfs -mkdir /user/stud22/ex2

22/05/18 18:43:07 INFO fs.TrashPolicyDefault: Moved: 'hdfs://bdl0.eng.tau.ac.il:8020/user/stud22/ex2' to trash at: hdfs://bdl0.eng.tau.ac.il:8020/user/stud22/.Trash/Current/user/stud22/ex21652888587384


put data and then show contents of folder

In [2]:
%%bash
hdfs dfs -put /home/stud22/workspace/ex2/microsoft-com.data/microsoft-com.data /user/stud22/ex2/microsoft-com.data
hdfs dfs -put /home/stud22/workspace/ex2/microsoft-com.data/countries.txt /user/stud22/ex2/countries.txt
hdfs dfs -ls /user/stud22/ex2

Found 2 items
-rw-r--r--   3 stud22 stud22        387 2022-05-18 18:43 /user/stud22/ex2/countries.txt
-rw-r--r--   3 stud22 stud22    1491520 2022-05-18 18:43 /user/stud22/ex2/microsoft-com.data


show last lines of data file

In [3]:
%%bash
hdfs dfs -cat /user/stud22/ex2/microsoft-com.data | tail -n 5

V,42709,1003,1
V,42710,1035,1
V,42710,1001,1
V,42710,1018,1
V,42711,1008,1


## 2) Parse the data according to the format specified in the .info file.

In [4]:
from pyspark import SparkContext, SparkConf

In [5]:
raw_rdd = sc.textFile("hdfs:/user/stud22/ex2/microsoft-com.data") # read data
lst_rdd = raw_rdd.map(lambda line: line.replace('"','').split(',')) # split each row by ,
lst_rdd.top(5)

[[u'V', u'42711', u'1008', u'1'],
 [u'V', u'42710', u'1035', u'1'],
 [u'V', u'42710', u'1018', u'1'],
 [u'V', u'42710', u'1001', u'1'],
 [u'V', u'42709', u'1003', u'1']]

## 3) only users that visited at least one page related to one of the given countries.
Given the list of countries in the file countries.txt (e.g., South Africa, Spain, Sweden, Switzerland), filter the data to include only users that visited at least one page related to one of the given countries.

**ASSUMPTION: the header is the same as the country name**

In [6]:
def filter_votes_by_countries(attr_rdd,votes_rdd,countries_rdd):
    # return votes for attributes which contains country name
    countries_lst = sc.broadcast(countries_rdd.collect()) # broadcast countries between nodes
    relevant_lines = attr_rdd.filter(lambda x: x[3] in countries_lst.value) # filter attribute lines which contains countries
    relevant_ids = relevant_lines.map(lambda x: x[1]) # get ids of relevant attribute line (which contains country)
    country_ids = sc.broadcast(relevant_ids.collect()) # TODO: verify that we don't cheat!! (might be lots of records)
    country_votes = votes_rdd.filter(lambda x: x[2] in country_ids.value) # return the votes
    return country_votes

In [7]:
attr_rdd = lst_rdd.filter(lambda x: x[0]=='A') # filter attribute lines
votes_rdd = lst_rdd.filter(lambda x: x[0]=='V') # filter votes lines
countries_rdd = sc.textFile("hdfs:/user/stud22/ex2/countries.txt")

In [8]:
attr_rdd.top(3)

[[u'A', u'1297', u'1', u'Central America', u'/centroam'],
 [u'A', u'1295', u'1', u'Training', u'/train_cert'],
 [u'A', u'1294', u'1', u'Bookshelf', u'/bookshelf']]

In [9]:
votes_rdd.top(3)

[[u'V', u'42711', u'1008', u'1'],
 [u'V', u'42710', u'1035', u'1'],
 [u'V', u'42710', u'1018', u'1']]

In [10]:
countries_rdd.top(3)

[u'Venezuela', u'Uruguay', u'UK']

In [11]:
country_votes = filter_votes_by_countries(attr_rdd,votes_rdd,countries_rdd)

In [12]:
country_votes.top(10)

[[u'V', u'42708', u'1123', u'1'],
 [u'V', u'42706', u'1059', u'1'],
 [u'V', u'42698', u'1005', u'1'],
 [u'V', u'42688', u'1053', u'1'],
 [u'V', u'42665', u'1023', u'1'],
 [u'V', u'42649', u'1223', u'1'],
 [u'V', u'42647', u'1023', u'1'],
 [u'V', u'42641', u'1053', u'1'],
 [u'V', u'42637', u'1053', u'1'],
 [u'V', u'42636', u'1053', u'1']]

In [13]:
relevant_users = country_votes.map(lambda x: x[1]).distinct() # drop duplicates

In [14]:
relevant_users.top(10)

[u'42708',
 u'42706',
 u'42698',
 u'42688',
 u'42665',
 u'42649',
 u'42647',
 u'42641',
 u'42637',
 u'42636']

In [15]:
print("number of relevant users: %s, number of relevant votes: %s"%(relevant_users.count(),country_votes.count()))

number of relevant users: 3199, number of relevant votes: 3334


In [16]:
# filter votes by the user ids we have found 
user_ids = sc.broadcast(relevant_users.collect()) 
relevant_data = votes_rdd.filter(lambda x: x[1] in user_ids.value)

In [17]:
relevant_data.top(5)

[[u'V', u'42708', u'1123', u'1'],
 [u'V', u'42708', u'1041', u'1'],
 [u'V', u'42708', u'1038', u'1'],
 [u'V', u'42708', u'1027', u'1'],
 [u'V', u'42708', u'1026', u'1']]

In [18]:
relevant_data.count()

13111

## 4.b) For each country, the number of users that visited a page of that country
For each country, the number of users that visited a page of that country. Exclude from your
report countries with a name longer than one word.

**ASSUMPTION: country names are splitted only by spaces**

**ASSUMPTION: titles are unique (validated on the last block)**

In [19]:
one_word_countries = countries_rdd.filter(lambda x: len(x.split(" ")) < 2) # get only one-word countries
one_word_lst = one_word_countries.collect() # small, constant size list - so it's fine to collect
country_name_mapping = dict(attr_rdd.filter(lambda x: x[3] in one_word_lst) \
.map(lambda x: (x[1],x[3])).collect()) # dict of attribute line id -> country name

In [20]:
print(country_name_mapping)

{u'1227': u'Argentina', u'1165': u'Poland', u'1166': u'Mexico', u'1084': u'UK', u'1223': u'Finland', u'1208': u'Israel', u'1080': u'Brazil', u'1262': u'Chile', u'1123': u'Germany', u'1267': u'Caribbean', u'1203': u'Denmark', u'1073': u'Taiwan', u'1209': u'Turkey', u'1217': u'Ireland', u'1053': u'Jakarta', u'1079': u'Australia', u'1059': u'Sweden', u'1195': u'Portugal', u'1194': u'China', u'1107': u'Slovakia', u'1112': u'Canada', u'1115': u'Hungary', u'1116': u'Switzerland', u'1258': u'Peru', u'1172': u'Belgium', u'1179': u'Colombia', u'1152': u'Russia', u'1229': u'Uruguay', u'1153': u'Venezuela', u'1188': u'Korea', u'1023': u'Spain', u'1183': u'Italy', u'1180': u'Slovenija', u'1241': u'India', u'1240': u'Thailand', u'1105': u'France', u'1005': u'Norway'}


In [21]:
country_votes_one = filter_votes_by_countries(attr_rdd,votes_rdd,one_word_countries) # relevant vote lines for one word countries

In [22]:
uid_vid = country_votes_one.map(lambda x: x[1:3]) # user id and attribute line id
uid_vid.top(5)

[[u'42708', u'1123'],
 [u'42706', u'1059'],
 [u'42698', u'1005'],
 [u'42688', u'1053'],
 [u'42665', u'1023']]

In [23]:
uid_country = uid_vid.map(lambda x: (x[0], country_name_mapping[x[1]])) # use the dict we created to get country name
uid_country.top(5)

[(u'42708', u'Germany'),
 (u'42706', u'Sweden'),
 (u'42698', u'Norway'),
 (u'42688', u'Jakarta'),
 (u'42665', u'Spain')]

In [24]:
uid_country_unique = uid_country.distinct() # count only once each user

In [25]:
countries = uid_country_unique.map(lambda x: x[1])
sum_by_country = countries.countByValue()
sum_by_country.items()

[(u'Brazil', 121),
 (u'Canada', 128),
 (u'Italy', 167),
 (u'Peru', 3),
 (u'France', 183),
 (u'Slovakia', 11),
 (u'Ireland', 13),
 (u'Caribbean', 5),
 (u'Argentina', 32),
 (u'Venezuela', 8),
 (u'Israel', 34),
 (u'Korea', 94),
 (u'Norway', 42),
 (u'Germany', 372),
 (u'Chile', 4),
 (u'Denmark', 55),
 (u'Belgium', 45),
 (u'Thailand', 11),
 (u'Poland', 38),
 (u'Spain', 191),
 (u'UK', 186),
 (u'Jakarta', 670),
 (u'Turkey', 9),
 (u'Finland', 29),
 (u'Sweden', 258),
 (u'Australia', 136),
 (u'Switzerland', 31),
 (u'Russia', 52),
 (u'Portugal', 15),
 (u'Mexico', 33),
 (u'Uruguay', 4),
 (u'India', 9),
 (u'China', 26),
 (u'Colombia', 11),
 (u'Hungary', 15),
 (u'Taiwan', 204),
 (u'Slovenija', 9)]

## 4.c) The top 5 visited countries.

In [26]:
country_votes = filter_votes_by_countries(attr_rdd,votes_rdd,countries_rdd)
country_votes_ids = country_votes.map(lambda x: x[2])
votes_count = country_votes_ids.countByValue() # returns a python dict - it's fine, because we deal with small dict
top_5_votes_count = sorted(votes_count.items(), key=lambda x:x[1], reverse=True)[:5]

In [27]:
print("The top 5 visited countries:")
for i in range(5):
    country_id, cnt = top_5_votes_count[i]
    print("%s. %s: %s"%(i+1,country_name_mapping[country_id],cnt))

The top 5 visited countries:
1. Jakarta: 670
2. Germany: 372
3. Sweden: 258
4. Taiwan: 204
5. Spain: 191


## 5) Write the report (4.a) to HDFS for future use

In [28]:
# TODO: based on 4.a which we skipped...
uid_vid.saveAsTextFile("hdfs:/user/stud22/ex2/output")

In [29]:
%%bash
hdfs dfs -ls /user/stud22/ex2/output

Found 3 items
-rw-r--r--   3 stud22 stud22          0 2022-05-18 18:43 /user/stud22/ex2/output/_SUCCESS
-rw-r--r--   3 stud22 stud22      33160 2022-05-18 18:43 /user/stud22/ex2/output/part-00000
-rw-r--r--   3 stud22 stud22      31920 2022-05-18 18:43 /user/stud22/ex2/output/part-00001


## *verify that titles are unique

In [30]:
countries_rdd = sc.textFile("hdfs:/user/stud22/ex2/countries.txt")
countries_lst = sc.broadcast(countries_rdd.collect())
tmp  = attr_rdd.filter(lambda x: x[3] in countries_lst.value).map(lambda x: x[3])
print("number of country titels: %s"%(tmp.count()))
print("number of distinct country titels: %s"%(tmp.distinct().count()))
assert(tmp.count() == tmp.distinct().count())
print("they are equal - so the assumption that the titles are unique holds...")

number of country titels: 41
number of distinct country titels: 41
they are equal - so the assumption that the titles are unique holds...
